In [9]:
"""Example usage:
  python object_detection/dataset_tools/create_oid_tf_record.py \
    --input_annotations_csv=/path/to/input/annotations-human-bbox.csv \
    --input_images_directory=/path/to/input/image_pixels_directory \
    --input_label_map=/path/to/input/labels_bbox_545.labelmap \
    --output_tf_record_path_prefix=/path/to/output/prefix.tfrecord
CSVs with bounding box annotations and image metadata (including the image URLs)
can be downloaded from the Open Images GitHub repository:
https://github.com/openimages/dataset
This script will include every image found in the input_images_directory in the
output TFRecord, even if the image has no corresponding bounding box annotations
in the input_annotations_csv.
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os

import contextlib2
import pandas as pd
import tensorflow as tf

from object_detection.dataset_tools import oid_tfrecord_creation
from object_detection.utils import label_map_util

In [10]:
tf.flags.DEFINE_string('input_annotations_csv', None,
                       'Path to CSV containing image bounding box annotations')
tf.flags.DEFINE_string('input_images_directory', None,
                       'Directory containing the image pixels '
                       'downloaded from the OpenImages GitHub repository.')
tf.flags.DEFINE_string('input_label_map', None, 'Path to the label map proto')
tf.flags.DEFINE_string(
    'output_tf_record_path_prefix', None,
    'Path to the output TFRecord. The shard index and the number of shards '
    'will be appended for each output shard.')
tf.flags.DEFINE_integer('num_shards', 100, 'Number of TFRecord shards')

FLAGS = tf.flags.FLAGS

DuplicateFlagError: The flag 'input_annotations_csv' is defined twice. First from /usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py, Second from /usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py.  Description from first occurrence: Path to CSV containing image bounding box annotations

In [18]:
input_annotations_csv='2017_07/validation/annotations-human-bbox.csv'
input_images_directory='raw_images_validation'
input_label_map='../object_detection/data/oid_bbox_trainable_label_map.pbtxt'
output_tf_record_path_prefix='test_tfrecords/test.tfrecord'
num_shards=100

In [19]:
tf.logging.set_verbosity(tf.logging.INFO)

required_flags = [
  'input_annotations_csv', 'input_images_directory', 'input_label_map',
  'output_tf_record_path_prefix'
]

label_map = label_map_util.get_label_map_dict(input_label_map)
all_annotations = pd.read_csv(input_annotations_csv)
all_images = tf.gfile.Glob(
  os.path.join(input_images_directory, '*.jpg'))
all_image_ids = [os.path.splitext(os.path.basename(v))[0] for v in all_images]
all_image_ids = pd.DataFrame({'ImageID': all_image_ids})
all_annotations = pd.concat([all_annotations, all_image_ids])

tf.logging.log(tf.logging.INFO, 'Found %d images...', len(all_image_ids))

IOError: [Errno 2] File 2017_07/validation/annotations-human-bbox.csv does not exist: '2017_07/validation/annotations-human-bbox.csv'